In [ ]:
#Thresholding

In [ ]:
%matplotlib inline

In [ ]:
#import libraries

In [ ]:
from PIL import Image
import numpy as np
from skimage import data
from skimage.morphology import disk
from matplotlib import pyplot as plt
from skimage.filters.rank import entropy
import math

In [ ]:
#loading images

In [ ]:
im1 = Image.open('onlineSamples/textures/1.jpg')
im2 = Image.open('onlineSamples/textures/2.jpg')
im3 = Image.open('onlineSamples/textures/3.jpg')

npimg1 = np.array(im1);
npimg2 = np.array(im2);
npimg3 = np.array(im3);
print(npimg1.shape,npimg2.shape,npimg3.shape);

In [ ]:
#renyi entropy

In [ ]:
import skimage.exposure as imexp

def renyi_seg_fn(im,alpha,coef):
    hist = imexp.histogram(im)
    
    #convert all values to float
    hist_float = [float(i) for i in hist[0]]
    
    #compute the pdf
    pdf = hist_float/np.sum(hist_float)
    
    #compute the cdf
    cumsum_pdf = np.cumsum(pdf)
    
    s = 0
    e = 255 # asuming 8 bit image
    scalar  = 1.0/(1-alpha)
    
    #A very small balue to prevent divition by zero
    eps = np.spacing(1)
    rr= e-s
    
    #the second parentheses is needed because the parameters are tuple
    h1 = np.zeros((rr,1))
    h2 = np.zeros((rr,1))
    
    #the following llop coputes h1 and h2 values used to compute the entropy
#     print (len(pdf),len(cumsum_pdf))
    for ii in range(1,rr):
           
        iidash = ii+s
        if(iidash==len(pdf)):   #temporatry fix for out of bound issue
            break
        temp1 = np.power(pdf[1:iidash]/cumsum_pdf[iidash],scalar)
        h1[ii] = np.log(np.sum(temp1)+eps)
        temp2 = np.power(pdf[iidash+1:255]/(1-cumsum_pdf[iidash]),scalar)
        h2[ii] = np.log(np.sum(temp2)+eps)
        
    T = h1 + h2
    
    #Entropy value is calculated 
    T = -T*scalar
    
    
    #location where the maximum entropy occurs is the threshold for the renyi entropy
    location = T.argmax(axis=0)
    
    sortedLocation = sorted(T.ravel())
    
#     print(max(sortedLocation))
#     print(sortedLocation[-3:])
#     print(sortedLocation[:2])
#     print(location)
    
    #show histogram
    
#     fig, axes= plt.subplots(1, 1, figsize=(10, 5), sharex=True, sharey=True)
    
#     axes.hist(location)
#     plt.show()
   
    
    #location value is used as the threshold
    thresh = location
    
#     print (thresh)
    threshedImg = im < thresh*coef
#     print ("done--")
    
    return threshedImg

In [ ]:
#addapted entropies

In [ ]:
from skimage.color.adapt_rgb import adapt_rgb, each_channel, hsv_value


#TODO SHOULD PLAY WITH THE STRUCTURE ELEMENT FOR DIFFERENT RESULTS


@adapt_rgb(each_channel)
def renyi_seg_each(image,alpha,coef):
    return renyi_seg_fn(image,alpha,coef)

@adapt_rgb(hsv_value)
def renyi_seg_hsv(image,alpha,coef):
    return renyi_seg_fn(image,alpha,coef)

In [ ]:
#adapted filters

In [ ]:
from skimage.color.adapt_rgb import adapt_rgb, each_channel, hsv_value
from skimage.filters import rank
from skimage.morphology import disk

selem = disk(20)

@adapt_rgb(each_channel)
def mean_percentile_each(image):
    return rank.mean_percentile(image, selem=selem, p0=.1, p1=.9)

@adapt_rgb(hsv_value)
def mean_percentile_hsv(image):
    return rank.mean_percentile(image, selem=selem, p0=.1, p1=.9)


@adapt_rgb(each_channel)
def mean_bilateral_each(image):
    return rank.mean_bilateral(image, selem=selem, s0=500, s1=500)

@adapt_rgb(hsv_value)
def mean_bilateral_hsv(image):
    return rank.mean_bilateral(image, selem=selem, s0=500, s1=500)


@adapt_rgb(each_channel)
def mean_each(image):
    return rank.mean(image, selem=selem)

@adapt_rgb(hsv_value)
def mean_hsv(image):
    return rank.mean(image, selem=selem)


@adapt_rgb(each_channel)
def median_each(image):
    return rank.median(image, selem=selem)

@adapt_rgb(hsv_value)
def median_hsv(image):
    return rank.median(image, selem=selem)


In [ ]:
#APPLYING TO SAMPLE IMAGES :BOTH HSV AND RGB 

In [ ]:
import math 

targetImg_titles = ['Original1','Original2','Original3']
srcImg = [npimg1,npimg2,npimg3]
displayImg = [npimg1,npimg2,npimg3]
myFiltersNames = ['entropy mean_each','entropy mean_hsv','entropy mean_percentile_each', 'entropy mean_percentile_hsv'
                  ,'entropy mean_bilateral_each','entropy mean_bilateral_hsv'
                  ,'entropy median_each','entropy median_hsv']
myEntropies = [renyi_seg_each]
myFilters = [mean_each,mean_hsv,mean_percentile_each,mean_percentile_hsv
             ,mean_bilateral_each,mean_bilateral_hsv,median_each,median_hsv]




 
###the higher the value of alpha the fewer the number of abnormal tissues detected
#alpha = 3
alpha = 5
###this coeficient is needed the detect the abonomal soft tissues
# coef = 0.01
coef = 1

#Dynamically prepare data for plotting
for n in range(0,len(myFilters)):
        for k in range(0,len(myEntropies)):
               for m in range(0,len(srcImg)):
                    displayImg.append(myEntropies[k](myFilters[n](srcImg[m]),alpha,coef))
                    targetImg_titles.append(myFiltersNames[n]+str(m+1)+":"+str(k+1))
            
         

#Dynamically plotting
mycols = len(srcImg)
myrows = int(math.ceil(float(len(displayImg))/float(mycols)))
print(myrows)
fig, axes= plt.subplots(myrows, mycols, figsize=(30, 8*myrows), sharex=True, sharey=True)
ax = axes.ravel()

for rrr in range(mycols*myrows):
    ax[rrr].imshow(displayImg[rrr],cmap='gray')
    ax[rrr].set_title(targetImg_titles[rrr])
    ax[rrr].axis('off')
    
plt.show()

In [ ]:
#HSV ONLY

In [ ]:
#Dynamically prepare data for plotting

def mass_plot(myFilters, myEntropies, srcImg,alpha, coef):
        for n in range(0,len(myFilters)):
                for k in range(0,len(myEntropies)):
                       for m in range(0,len(srcImg)):
                            displayImg.append(myEntropies[k](myFilters[n](srcImg[m]),alpha,coef))
                            targetImg_titles.append(myFiltersNames[n]+str(m+1)+":"+str(k+1))



        #Dynamically plotting
        mycols = len(srcImg)
        myrows = int(math.ceil(float(len(displayImg))/float(mycols)))
#         print(myrows)
        fig, axes= plt.subplots(myrows, mycols, figsize=(30, 8*myrows), sharex=True, sharey=True)
        ax = axes.ravel()

        for rrr in range(mycols*myrows):
            ax[rrr].imshow(displayImg[rrr],cmap='gray')
            ax[rrr].set_title(targetImg_titles[rrr])
            ax[rrr].axis('off')

        plt.show()

In [ ]:
import math 

targetImg_titles = ['Original1','Original2','Original3']
srcImg = [npimg1,npimg2,npimg3]
displayImg = [npimg1,npimg2,npimg3]
myFiltersNames = ['entropy mean_hsv', 'entropy mean_percentile_hsv'
                  ,'entropy mean_bilateral_hsv','entropy median_hsv']
myEntropies = [renyi_seg_each]
myFilters = [mean_hsv,mean_percentile_hsv
             ,mean_bilateral_hsv,median_hsv]




 
###the higher the value of alpha the fewer the number of abnormal tissues detected
#alpha = 3
###this coeficient is needed the detect the abonomal soft tissues
# coef = 0.01
coef = 1

for alpha in range(3, 100):
    print "MASS PLOATING-----" + str(alpha/float(2))
    mass_plot(myFilters, myEntropies, srcImg,alpha/float(2),coef)
  

